## Project: AI-BEMS 알고리즘 도출
* 2024.05.21. SQI에서 인천광역시 고등학교 17개소의 BEMS 데이터 공유 (현재 작업 중)
* 2024.10.11. SQI에서 2023.1. ~ 2024.10. 까지 BEMS 데이터 업데이트

In [2]:
# 라이브러리를 수정할 때마다 autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import pickle

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/buildingID_devID_map.csv"
buildingID_devID_df = pd.read_csv(file_path)
print(buildingID_devID_df.head(3))

file_path = r"/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/devID_tagCD_map.csv"
devID_tagCD_df = pd.read_csv(file_path)
print(devID_tagCD_df.head(3))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  buildingID  devID                    devName devType
0      B0001   1001     인천해양과학고등학교 인버터1_2.53kW     인버터
1      B0001   1002     인천해양과학고등학교 인버터2_2.53kW     인버터
2      B0001   1003  인천해양과학고등학교 신축인버터1_49.95kW     인버터
       devID_range devID_description  tagCD         tagName
0  devID 1001~1999   inverter 태양광인버터  10002           PV 전압
1  devID 1001~1999   inverter 태양광인버터  10003  계통 R상(RS선간) 전압
2  devID 1001~1999   inverter 태양광인버터  10004  계통 S상(ST선간) 전압


In [ ]:
import numpy as np
import pandas as pd
import Utility as Util

#########
buildingID, buildingName = 'B0001', '해양과학고'
buildingID, buildingName = 'B0010', '교육연수원'
file_path = '/home/ymatics/CodingSpace/2024_AI_BEMS/241014_data_colec_h/data_colec_h_202410141150_B0010.csv'
#########
df_buildID = pd.read_csv(file_path)
df_buildID['colec_dt'] = pd.to_datetime(df_buildID['colec_dt']).dt.floor('min')  # 분 이하는 제거
print(f"Start timestamp: first: {df_buildID['colec_dt'].iloc[0]}, min: {df_buildID['colec_dt'].min()}")
print(f"End timestamp: last: {df_buildID['colec_dt'].iloc[-1]}, max: {df_buildID['colec_dt'].max()}")

dev_id_list = df_buildID['dev_id'].unique()
print(f"{len(dev_id_list)=}")
Inverter = np.sum((dev_id_list >= 1000) & (dev_id_list < 2000))
ElecMeter = np.sum((dev_id_list >= 2000) & (dev_id_list < 3000))
GasMeter = np.sum((dev_id_list >= 3000) & (dev_id_list < 4000))
HeatMeter = np.sum((dev_id_list >= 4000) & (dev_id_list < 5000))
EnvSensor = np.sum((dev_id_list >= 5000) & (dev_id_list < 6000))
AirCon = np.sum((dev_id_list >= 6000) & (dev_id_list < 7000))
print(f'{Inverter=}, {ElecMeter=}, {GasMeter=}, {HeatMeter=}, {EnvSensor=}, {AirCon=}')
df_buildID.head(3)

### 임시로 그래프 보기 위해서...
start_date, end_date = pd.to_datetime('2023-01-01 00:00:00'), pd.to_datetime('2024-10-14 10:00:00')
df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]
#devID==2001 "학교MAIN", tagCD==30001 "현재출력", peak_prediction()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2001]))])

devID, device_name = 2001, '학교MAIN'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'
#tags[40007] = calc_tagCD_diff(tags[40007])  # '역송전력량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True)

### 기간 설정

In [10]:
start_date, end_date = pd.to_datetime('2023-01-01 00:00:00'), pd.to_datetime('2024-10-14 10:00:00')
start_date, end_date = pd.to_datetime('2023-01-01 00:00:00'), pd.to_datetime('2023-03-01 10:00:00')

df = df_buildID[(df_buildID['colec_dt'] >= start_date) & (df_buildID['colec_dt'] <= end_date)]

### 인버터

In [ ]:
#devID==1001(x) 1002(x) 1003 1004 "인버터", tagCD==30001 "현재출력"
#anomaly_detection()
#missing_value_interpolation()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([1001, 1002, 1003, 1004]))])

devID, device_name = 1004, '인버터'
Util.print_tagCD(df, devID)
tag_dict = {10002: 'PV전압', 20002: 'PV전류', 30001: '현재출력', 30002: 'PV출력', 60001: '주파수', 70006: '인버터온도', 202202: 'validData'}
tag_dict = {30001: '현재출력'}

tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#df_raw = pd.DataFrame(data=tag_data[0]['colec_val'].values, index=tag_data[0]['colec_dt'], columns=['colec_val'])
#df_interpol = Util.resample_time_index_interpolate_NaN_df(df_raw, '15min', 'linear')
#Util.plot_interpolated_data(df_raw, df_interpol, device_name, createFig=False)


#### 학교 MAIN

In [13]:
#devID==2001 "학교MAIN", tagCD==30001 "현재출력", peak_prediction()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2001]))])

devID, device_name = 2001, '학교MAIN'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량_차', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'
#tags[40007] = calc_tagCD_diff(tags[40007])  # '역송전력량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)


  buildingID  devID  devName devType
4      B0001   2001  학교 MAIN   전기계측기
tagCD list:  [202202  30001  40004  40005  50001]


#### EHP

In [13]:
#devID==2002 2044 2072 2116 2118 2119 "교사동EHP" "체육관EHP" "실습2동 EHP" "한나루관 EHP" "식당 EHP1" "식당 EHP2", tagCD==30001 "현재 출력", anomaly_detection()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2002, 2044, 2072, 2116, 2118, 2119]))])

devID, device_name = 2118, 'EHP'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]
Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

    buildingID  devID       devName devType
5        B0001   2002       교사동 EHP   전기계측기
47       B0001   2044  체육관 EHP MAIN   전기계측기
75       B0001   2072      실습2동 EHP   전기계측기
119      B0001   2116      한나루관 EHP   전기계측기
121      B0001   2118       식당 EHP1   전기계측기
122      B0001   2119       식당 EHP2   전기계측기
tagCD list:  [202202  30001  40004  50001]


#### 전기 계측기

In [ ]:
#devID==2003 2004 2005 2006 2020 2061 2085 2117 "교사동EL" "실습1동 MAIN" "실습2동 MAIN" "교사서측1F MAIN" "교사중앙1F MAIN" "체육관 MAIN" "한나루관 MAIN" "식당 MAIN", tagCD==30001 "현재 출력", anomaly_detection()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2003, 2004, 2005, 2006, 2020, 2061, 2085, 2117]))])

devID, device_name = 2003, '교사동EL'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True)

#### 전기온수기

In [14]:
#이상진단
#devID==2010 2024~2026 2030~2032 2036~2038 2042~2043 2045 2063 2067 2071 2075~2077 2081 "전기온수기", tagCD==30001 "현재 출력", anomaly_detection()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2010, 2024, 2025, 2026, 2030, 2031, 2032, 2036, 2037, 2038, 2042, 2043, 2045, 2063, 2067, 2071, 2075, 2076, 2077, 2081]))])

devID, device_name = 2081, '전기온수기'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

   buildingID  devID          devName devType
13      B0001   2010   교사 서측 1F 전기온수기   전기계측기
27      B0001   2024  교사 중앙 1F 전기온수기1   전기계측기
28      B0001   2025  교사 중앙 1F 전기온수기2   전기계측기
29      B0001   2026  교사 중앙 1F 전기온수기3   전기계측기
33      B0001   2030  교사 중앙 2F 전기온수기1   전기계측기
34      B0001   2031  교사 중앙 2F 전기온수기2   전기계측기
35      B0001   2032  교사 중앙 2F 전기온수기3   전기계측기
39      B0001   2036  교사 중앙 3F 전기온수기1   전기계측기
40      B0001   2037  교사 중앙 3F 전기온수기2   전기계측기
41      B0001   2038  교사 중앙 3F 전기온수기3   전기계측기
45      B0001   2042  교사 중앙 4F 전기온수기1   전기계측기
46      B0001   2043  교사 중앙 4F 전기온수기2   전기계측기
48      B0001   2045     체육관 급탕 전기온수기   전기계측기
66      B0001   2063    실습1동 1F 전기온수기   전기계측기
70      B0001   2067    실습1동 2F 전기온수기   전기계측기
74      B0001   2071    실습1동 3F 전기온수기   전기계측기
78      B0001   2075   실습2동 1F 전기온수기1   전기계측기
79      B0001   2076   실습2동 1F 전기온수기2   전기계측기
80      B0001   2077   실습2동 1F 전기온수기3   전기계측기
84      B0001   2081    실습2동 2F 전기온수기   전기계측기
tagCD list:  [202202  30001  40004

#### 전등

In [ ]:
#devID==2046~2050 2055~2060 2062 2086~2115 2124~2126 "전등", tagCD==30001 "현재 출력", anomaly_detection()
#devID==2007~2009 2011~2013 2014~2016 2017~2019 2021~2023 2024~2029 2033~2035 2039~2041 2064~2066 2068~2070 2073~2074 2078~2080 2082~2084 "전등/전열교환기/선풍기", tagCD==30001 "현재 출력", anomaly_detection()

devID, device_name = 2062, '전등'
Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 환경 센서

In [ ]:
#devID==5001 5002 5003 5004 5005 5006 5007 "온도/습도/CO2/초미세먼지/미세먼지/조도", tagCD==80002, "온도" anomaly_detection()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([5001, 5002, 5003, 5004, 5005, 5006, 5007]))])

devID, device_name = 5003, '온도/습도/CO2/초미세먼지/미세먼지/조도'
Util.print_tagCD(df, devID)
tag_dict = {80002: '온도', 80003: '습도', 80004: 'CO2', 80005: '미세먼지', 80006: '초미세먼지', 80007: '조도', 202202: 'validData'}
tag_dict = {80002: '온도'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 배기휀

In [ ]:
### 이상진단 ###
#devID==2051~2054 2120~2123 "배기휀", tagCD==30001 "현재 출력", anomaly_detection()   
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([2051, 2052, 2053, 2054, 2120, 2121, 2122, 2123]))])

devID, device_name = 2051, '체육관 B1F 탁구장 배기휀   전기계측기'  # 작동없음
devID, device_name = 2052, '체육관 B1F 탁구장 배기휀   전기계측기'  # 작동없음
devID, device_name = 2053, '체육관 B1F 전기실 급기휀   전기계측기'  # 작동없음
devID, device_name = 2054, '체육관 B1F 전기실 배기휀   전기계측기'  # 작동없음
devID, device_name = 2120, '식당 급기휀   전기계측기'  # 작동없음
devID, device_name = 2121, '식당 배기휀1   전기계측기'  # 정상 패턴
devID, device_name = 2122, '식당 배기휀2   전기계측기'  # 이상 패턴
devID, device_name = 2123, '식당 배기휀3   전기계측기'  # 정상 패턴

Util.print_tagCD(df, devID)
tag_dict = {30001: '현재 출력', 40004: '전체누적사용량_차', 40007: '역송전력량', 50001: '역률', 202202: 'validData'}
tag_dict = {30001: '현재 출력'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[40004] = calc_tagCD_diff(tags[40004])  # '전체누적사용량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#### 전기온수기 급탕

In [ ]:
### 이상진단 ###
#devID==4001 4002 4003 "전기온수기 급탕", tagCD==90003 "순시유량", anomaly_detection()
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([4001, 4002, 4003]))])

devID, device_name = 4003, '체육관 전기온수기 급탕'
Util.print_tagCD(df, devID)
tag_dict = {90001: '공급온도', 90002: '환수온도', 90003: '순시유량', 90004: '누적유량_차', 90005: '순시열량', 202202: 'validData'}
tag_dict = {90003: '순시유량'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = Util.calc_tagCD_diff(tags[90004])
#tags[90006] = calc_tagCD_diff(tags[90006])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=False)

#df_raw = pd.DataFrame(data=tag_data[0]['colec_val'].values, index=tag_data[0]['colec_dt'], columns=['colec_val'])
#df_interpol = Util.resample_time_index_interpolate_NaN_df(df_raw, '15min', 'linear')
#Util.plot_interpolated_data(df_raw, df_interpol, device_name, createFig=False)

#### 가스 계측기

In [11]:
#devID==3001 3002 3003 3004 "누적가스량", tagCD==70002 '보정적산량_차', anomaly_detection(), vacation_detection()~50% 선
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([3001, 3002, 3003, 3004]))])

devID, device_name = 3001, '누적가스량'
Util.print_tagCD(df, devID)
tag_dict = {70002: '보정적산량_차'}
tag_dict = {key: f"{value}@{device_name}" for key, value in tag_dict.items()}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
tags[70002] = Util.calc_tagCD_diff(tags[70002])  # '보정적산량_차'

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True)

     buildingID  devID  devName devType
1102      B0010   3001  본관GHP가스   가스계측기
1103      B0010   3002   별관급탕가스   가스계측기
1104      B0010   3003   별관식당가스   가스계측기
1105      B0010   3004  별관GHP가스   가스계측기
tagCD list:  [202202  70002]


In [16]:
for i, data in enumerate(tag_data):
    print(f"\nData {i}:")
    print("colec_val values:")
    for val in data['colec_val']:
        print(val)


Data 0:
colec_val values:
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.3000000000029104
0.2999999999956344
0.3000000000029104
0.4000000000014552
0.2999999999956344
0.3000000000029104
0.3000000000029104
0.39999999999417923
0.20000000000436557
0.4000000000014552
0.2999999999956344
0.20000000000436557
0.2999999999956344
0.3000000000029104
0.2999999999956344
0.20000000000436557
0.2999999999956344
0.3000000000029104
0.19999999999708962
0.20000000000436557
0.4000000000014552
0.2999999999956344
0.20000000000436557
0.2999999999956344
0.3000000000029104
0.19999999999708962
0.3000000000029104
0.2999999999956344
0.3000000000029104
0.3000000000029104
0.5999999999985448
0.6999999999970896
0.8000000000029104
0.5999999999985448
0.6999999999970896
0.7000000000043656
0.6999999999970896
0.7000000000043656
0.6999999999970896
0.6999999999970896
0.6000000000058208
0.6999999999970896
0.6999999999970896
0.7000000000

#### 에어컨

In [ ]:
# 관찰 구간
#devID==6001~6083, 6085~6149 "에어컨", tagCD==?????? '??????', anomaly_detection(), vacation_detection(), optimal_control()
#100001	운전선택탭  off: 0, on: 1  <-- 이산치인데 데이터누락이 많아서 정보가치 없음
#100002	냉난방모드	cool: 0, heat: 1, dry: 2, fan: 3, auto: 4, normal: 5, env: 6, sleep: 6  <-- 사용하지 않는 듯
#100003	현재온도	현재온도
#100004	희망온도	냉방 18.0~30.0, 난방 16.0~30.0
#100005	풍량        low: 0, med: 1, mid: 1, high: 2, auto: 3
#100006	리모컨허용  level: 2, true: 1, false: 0
#100007	에러	    indoor: 0, outdoor: 1, caur: 2, trans: 3
print(buildingID_devID_df[(buildingID_devID_df['buildingID'] == buildingID) & (buildingID_devID_df['devID'].isin([6001, 6002, 6003, 6004]))])

devID, device_name = 6020, '에어컨'
Util.print_tagCD(df, devID)
tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도', 100005: '풍량', 100006: '리모컨허용', 100007: '에러', 202202: 'validData'}
# 
tag_dict = {100001: '운전선택탭', 100002: '냉난방모드', 100003: '현재온도', 100004: '희망온도'}
# dictionary를 이용해 각 태그 데이터를 선택
tags = {key: Util.select_devID_tagCD(df, devID, tagCD=key) for key in tag_dict.keys()}
# 특정 태그에 대해 추가 연산 수행 (예: 전체누적사용량 차이 계산)
#tags[90004] = calc_tagCD_diff(tags[90004])

# 그래프를 그리기 위해 필요한 데이터를 리스트로 변환
tag_data = [tags[key] for key in tag_dict.keys()]
tag_names = [tag_dict[key] for key in tag_dict.keys()]

Util.plot_dfL_devID_tagCD(tag_data, tag_names, device_name, createFig=True, W=None, H=600)

In [22]:
tag_data[0]

,dev_id,tag_cd,colec_dt,colec_val,reg_dt,bldg_id
1423,6020,100001,2023-01-01 00:15:00,0.0,2023-01-01 00:24:14.821,B0001
3161,6020,100001,2023-01-01 00:45:00,0.0,2023-01-01 01:07:14.986,B0001
4899,6020,100001,2023-01-01 01:15:00,0.0,2023-01-01 01:30:15.213,B0001
7319,6020,100001,2023-01-01 02:00:00,0.0,2023-01-01 02:12:15.518,B0001
9057,6020,100001,2023-01-01 02:30:00,0.0,2023-01-01 02:45:15.361,B0001
...,...,...,...,...,...,...
41594679,6020,100001,2024-10-14 07:45:00,0.0,2024-10-14 07:55:52.535,B0001
41596721,6020,100001,2024-10-14 08:15:00,0.0,2024-10-14 08:19:53.064,B0001
41598763,6020,100001,2024-10-14 08:45:00,0.0,2024-10-14 09:19:52.825,B0001
41600805,6020,100001,2024-10-14 09:15:00,0.0,2024-10-14 09:19:52.971,B0001
